In [42]:
import networkx as nx
import matplotlib.pyplot as plt
from surface_dynamics.databases.flat_surfaces import CylinderDiagrams
from surface_dynamics import AbelianStratum

In [3]:
def check_sets(element, list_sets):
    for s in list_sets:
        if element in s:
            return True
    return False

def compute_singularities(perm):
    """Computes a list of singularities. 
    
    Each saddle is identified with its right vertex. A singularity is then an equivalence class (set) of saddles."""
    singularities = []
    for i in perm:
        if check_sets(i, singularities):
            continue
        
        next_set = {i}
        while perm[i] not in next_set:
            i = perm[i]
            next_set.add(i)
        singularities.append(next_set)
    return singularities

def find_element_in_partition(partition, element):
    """Gives the singularity corresponding to the right vertex of the saddle."""
    for i, part in enumerate(partition):
        if element in part:
            return i

def collapse_same_genus(cylinder_diagram, cylinder_class):
    """Tells whether a certain collapse will be in the same genus.
    
    Return true if the collapse is forced to be the same genus.
    Return false if the collapse may be a lower genus.
    
    cylinder_list is a list of ints corresponding to the cylinder equivalence class being collapsed
    cylinder_diagram is a surface_dynamics.flat_surfaces.separatrix_diagram.CylinderDiagram object that represents a translation surface
    """
    sing_list = compute_singularities(cylinder_diagram.outgoing_edges_perm())
    digraph = nx.DiGraph()
    digraph.add_nodes_from(range(len(sing_list)))
    
    cylinders = cylinder_diagram.cylinders()
    for cylinder in cylinder_class:
        for c_index in cylinder:
            for bot_saddle in cylinders[c_index][0]:
                for top_saddle in cylinders[c_index][1]:
                    start = find_element_in_partition(sing_list, bot_saddle)
                    end = find_element_in_partition(sing_list, top_saddle)
                    digraph.add_edge(start, end)
    # nx.draw(digraph, with_labels=True, font_weight='bold')
    if nx.is_directed_acyclic_graph(digraph):
        return True
    return False

In [39]:
def find_generic_pants(digraph: nx.DiGraph):
    """Finds cases when n cylinders are all complete attached to the side of a
    single cylinder.
    
    Input: A digraph representation of cylinder adjacencies of a horizontally
    periodic translation surface. The surface should have more than one
    cylinder.
    
    Output: A list of lists repsenting homology restrictions."""
    output = []
    for n in digraph:
        successors = list(digraph.successors(n))
        if all([list(digraph.predecessors(suc)) == [n] for suc in successors]):
            successors.append(n)
            output.append(successors)

        predecessors = list(digraph.predecessors(n))
        if all([list(digraph.successors(pre)) == [n] for pre in predecessors]):
            predecessors.append(n)
            # check if this relation already exists
            if not set(successors) == set(predecessors):
                output.append(predecessors)
    return output

G = nx.DiGraph()
G.add_edge(0, 0)
G.add_edge(0, 1)
G.add_edge(1, 0)
assert(find_generic_pants(G) == [])
G = nx.DiGraph()
G.add_edge(0, 1)
G.add_edge(0, 2)
G.add_edge(1, 0)
G.add_edge(2, 0)
output = find_generic_pants(G)
assert(len(output) == 1 and set(output[0]) == {0, 1, 2})
G = nx.DiGraph()
G.add_edge(0, 1)
G.add_edge(0, 2)
G.add_edge(0, 3)
G.add_edge(1, 0)
G.add_edge(2, 0)
G.add_edge(3, 0)
output = find_generic_pants(G)
assert(len(output) == 1 and set(output[0]) == {0, 1, 2, 3})

def check_conditions(partition, condition_list):
    """Check the partition satisfies a property.
    
    condition is a list of integers. 
    Either one set must contain all integers in the list, or they must be split
    among at least 3 of the sets.
    """
    for condition in condition_list:
        partition_sets = map(
            lambda c: find_element_in_partition(partition, c),
            condition
        )
        if len(set(partition_sets)) == 2:
            return False
    return True

assert(not check_conditions([{0, 1}, {2, 3}], [[1, 2, 3]]))
assert(check_conditions([{1}, {2}, {3}], [[1, 2, 3]]))


def partitions(n, m, singletons=True):
    """List all ways to partition the set [1..n] into m sets.
    
    If singletons==False, do not allow singleton sets."""
    def contains_singleton(l):
        return any([i == 1 for i in l])

    partitions = []
    underlying_part = Partitions(n, length=m).list()
    if not singletons:
        underlying_part = [i for i in underlying_part if not contains_singleton(i)]
    for up in underlying_part:
        partitions.extend(SetPartitions(range(n), up))
    return partitions

assert(len(partitions(5, 2, singletons=False)) == 10)
assert(len(partitions(5, 2, singletons=True)) == 15)
assert(len(partitions(6, 2, singletons=False)) == 25)
assert(len(partitions(6, 3, singletons=False)) == 15)

def valid_cylinder_equivalence_classes(cyl_diag, num_classes, free_cylinders = True):
    """Cylinders are numbered. An edge is draw if the cylinders are connected."""
    cylinders = cyl_diag.cylinders()
    
    digraph_data = [[None, None] for _ in range(cyl_diag.degree())]
    for i, (bot, top) in enumerate(cyl_diag.cylinders()):
        for separatrix in bot:
            digraph_data[separatrix][0] = i
        for separatrix in top:
            digraph_data[separatrix][1] = i
    
    G = nx.DiGraph()
    G.add_nodes_from(range(len(cylinders)))
    for source, dest in digraph_data:
        G.add_edge(source, dest)
    # nx.draw(G, with_labels=True, font_weight='bold')
    
    relations = find_generic_pants(G)
    part = partitions(len(cylinders), num_classes, free_cylinders)
    part = [p for p in part if check_conditions(p, relations)]
    return part

In [91]:
C = CylinderDiagrams()
# H = AbelianStratum(2, 2).components()[1]
# H = AbelianStratum(2, 1, 1).components()[0]
H = AbelianStratum(1, 1, 1, 1).components()[0]


# H = AbelianStratum(2, 2).components()[1]
num_cylinders = 4
num_classes = 2


diagram_list = list(C.get_iterator(H, num_cylinders))
print(len(diagram_list))


27


In [92]:
i = 0
for cd in C.get_iterator(H, num_cylinders):
    possible_classes = valid_cylinder_equivalence_classes(cd, num_classes, free_cylinders=False)
    if possible_classes:
        print(cd)
        i += 1
        # print('\t', compute_singularities(cd.outgoing_edges_perm()))
print(i)

(0,3)-(0,5) (1,2)-(1,4) (4,6)-(3,7) (5,7)-(2,6)
(0,7)-(5,6) (1,5)-(2,7) (2,4)-(1,3) (3,6)-(0,4)
(0,5,1)-(6,7) (2,6)-(5) (3,7)-(0,4,1) (4)-(2,3)
(0,1,4)-(0,1,7) (2,3)-(2,6) (5,6)-(4) (7)-(3,5)
(0,4,2)-(3,7) (1,3)-(1,5) (5,7)-(0,6,2) (6)-(4)
(0,2,4)-(0,2,7) (1,6,3)-(4) (5)-(6) (7)-(1,5,3)
(0,3,5)-(0,3,7) (1,2,4)-(1,2,6) (6)-(5) (7)-(4)
(0,5,2)-(1,7,3) (1,4,3)-(0,6,2) (6)-(5) (7)-(4)
(0,5,2)-(1,7,3) (1,4,3)-(0,6,2) (6)-(4) (7)-(5)
(0,7,2)-(1,6,3) (1,4,3)-(7) (5)-(4) (6)-(0,5,2)
10


In [88]:
C.default_path

'/home/duohead2/.sage/local/lib/python3.10/site-packages/surface_dynamics/databases'